In [1]:
from html.parser import HTMLParser
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [2]:
num = input("How many stocks would you like in your portfolio?")
print("Enter the ticker symbol of these stocks")

tkrs = np.array([])

for i in range(int(num)):
    tkrs = np.append(tkrs, input())

print(tkrs)

How many stocks would you like in your portfolio? 2


Enter the ticker symbol of these stocks


 EHTH
 VWDRY


['EHTH' 'VWDRY']


1756

In [3]:
pages = np.array([])

for tkr in tkrs:
    pages = np.append(pages, requests.get('https://finance.yahoo.com/quote/' + tkr +'/history?p=' + tkr))


#page = requests.get('https://finance.yahoo.com/quote/EHTH/history?p=EHTH')
#print(page)
#url = 'https://finance.yahoo.com/quote/EHTH/history?p=EHTH'

In [4]:
print(pages)

[<Response [200]> <Response [200]> <Response [200]>]


In [4]:
soups = np.array([])

for page in pages:
    soups = np.append(soups, BeautifulSoup(page.content))

#soup = BeautifulSoup(page.content)
#print(soup)

In [5]:
#clean numpy array of 'html' string to avoid errors later

for i in range(len(soups)-2):
    if soups[i] == "html":
        soups = np.delete(soups, i)


In [30]:
#repos = np.array([])

#repos = np.append(repos, list(soups[1].find_all("td", "Py(10px)")))

#for soup in soups:
#    repos = np.append(repos, soup.find_all("td", "Py(10px)"))

#repo = soup.find_all("td", "Py(10px)")
#print(len(repo))

<td class="Py(10px) Ta(start) Pend(10px)" data-reactid="50"><span data-reactid="51">Dec 02, 2019</span></td>


Empty DataFrame
Columns: []
Index: []


In [6]:
dataframe = pd.DataFrame()

for soup in soups:
    lst = np.array([])
    m = 3
    repo = soup.find_all("td", "Py(10px)")
    for item in repo:
        if m % 7 == 0:
            lst = np.append(lst, item.text)
        m = m + 1
    series = pd.Series(lst)
    dataframe = dataframe.append(series, ignore_index = True)

In [7]:
data = dataframe.transpose()
data.columns = tkrs
print(data)

      EHTH  VWDRY
0    95.01  33.84
1    93.69  33.95
2    92.85  32.78
3    92.65  32.39
4    91.22  32.61
..     ...    ...
95  103.75  27.44
96  105.50  27.49
97  105.84  29.28
98  108.40  29.25
99   85.48  29.36

[100 rows x 2 columns]


In [8]:
for tkr in tkrs:
    data[tkr] = pd.to_numeric(data[tkr])

In [10]:
print(data.EHTH)

0      95.01
1      93.69
2      92.85
3      92.65
4      91.22
       ...  
95    103.75
96    105.50
97    105.84
98    108.40
99     85.48
Name: EHTH, Length: 100, dtype: float64


In [11]:
stdev = np.array([])
for tkr in tkrs:
    stdev = np.append(stdev, data[tkr].std())

In [12]:
print(stdev)

[16.02504107  2.57428426]


In [13]:
corrs = np.array([])

for i in range(len(tkrs)):
    for j in range(len(tkrs)):
        corrs = np.append(corrs, data[tkrs[i]].corr(data[tkrs[j]]))

In [14]:
print(corrs)

[1.         0.23937472 0.23937472 1.        ]


In [15]:
#create covariance matrix
cov = np.array([[]])

for i in range(len(tkrs)):
    for j in range(len(tkrs)):
        cov = np.append(cov, data[tkrs[i]].cov(data[tkrs[j]]))
    

In [16]:
print(cov)

[256.80194117   9.8749278    9.8749278    6.62693943]


In [18]:
print(data[tkrs[0]].cov(data[tkrs[0]]))

256.8019411717172


In [19]:
exp_return = np.array([])

for tkr in tkrs:
    exp_return = np.append(exp_return, (data.iloc[0][tkr]-data.iloc[99][tkr]) / data.iloc[99][tkr])

In [121]:
A = np.array([1.2])
B = np.array([3,4])
print(sum(A*B))

8.399999999999999


In [99]:
from scipy.optimize import Bounds
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint

In [155]:
#w is the array of weights, r is the array of expected returns
def expected_return(x):
    return (sum(x[1:]*exp_return[1:]))

In [184]:
w = np.array([1,2])
r = np.array([3,4])
print(w*r)

[3 8]


In [181]:
risk = input("What amount of risk (variance) are you willing to take?")

What amount of risk (variance) are you willing to take? 5.0


In [133]:
bounds = Bounds(-1,1)
#constraint = NonlinearConstraint()

In [185]:
bounds = Bounds(50,100)
def constr1(x):
    return (((x[1:].transpose()).dot(cov)).dot(x[1:])-risk))

def constr2(x):
    return np.array([x[0]+x[1]-1])

x0 = [1,1]
con = [{'type': 'eq', 'fun': constr1}, {'type':'eq', 'fun': constr2}]
res = minimize(expected_return, x0, bounds=bounds, method='SLSQP', constraints = con, options = {'disp':True})

SyntaxError: invalid syntax (<ipython-input-185-6b88cdc06384>, line 3)

In [56]:
#def rosen(x):
#    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

#x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
#res = minimize(rosen, x0, method = 'nelder-mead', options = {'xtol':1e-8, 'disp':True})

In [171]:
print(res.x)

[50. 50.]
